In [7]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import backtrader as bt
import backtrader.feeds as btfeeds
from pathlib import Path
import datetime
import csv

from workspace.lib.utils import construct_csv_path, write_to_csv, merge_csv
from workspace.lib.scraper import get_candles, get_exchange

exchange_id = 'ftx'
pair = 'BTC-PERP'
timeframe = '5m'

base_path = Path('../data/')
full_path = construct_csv_path(base_path, exchange_id, pair, timeframe)

data = btfeeds.GenericCSVData(
    dataname=full_path,
    timeframe=bt.TimeFrame.Minutes, compression=5,  # for 5 min candles?
    dtformat=lambda x: datetime.datetime.utcfromtimestamp(int(x) // 1000),
    openinterest=-1
)

def _get_data(from_datetime, to_datetime):
    exchange = get_exchange(exchange_id)
    candles = get_candles(exchange, pair, timeframe, from_datetime, to_datetime, logger=print)
    return candles

def get_and_merge_data(from_datetime, to_datetime):
    candles = _get_data(from_datetime, to_datetime)
    print(candles[-1][0])
    merge_csv(full_path, candles)
    return candles

def get_and_overwrite_data(from_datetime, to_datetime):
    candles = _get_data(from_datetime, to_datetime)
    write_to_csv(full_path, candles)
    return candles


In [8]:
def runStrategy(data, stratClass, params):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(stratClass, **params)
    cerebro.adddata(data)
    cerebro.broker.setcash(10000.0)
    cerebro.broker.setcommission(commission=0.0007)

    print('Starting portfolio value: %.2f' % (cerebro.broker.getvalue()))
    cerebro.run()
    print('Final value: %.2f' % (cerebro.broker.getvalue()))

    return cerebro  # do what you want with it eg plot

    

In [9]:
class NoopStrat(bt.Strategy):
    # Strat class that only logs and passes params
    params = (
        ('minutes', 0),
    )
    
    def __init__(self):
        print('Logging every %s minutes' % self.params.minutes)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def next(self):
        time = self.datas[0].datetime.time(0)
        if time.minute % self.params.minutes == 0:
            self.log('Close: %s' % self.datas[0].close[0])
        

In [ ]:
cerebro = runStrategy(data, NoopStrat, {
    'minutes': 60
})  # Todo: limit to only certain time

## Strategy testing

Define a function that takes a strategy and runs it across several backtest timeframes sequentially

First we ensure we have enough data in our feed. Naive for now -- does not take into account different timeperiods in the year, just a blanket X days before the present

In [14]:
test_period_in_days = 31
num_periods = 8
offset_in_days = 31 * 3  # leave 3 months in front for possible walk forward analysis?

_delta = datetime.timedelta(days=test_period_in_days * num_periods + offset_in_days)
earliest_required_timestamp = datetime.datetime.now() - _delta
oldest_available_timestamp = None

# Read first line of csv and check if its earlier than the earliest required date
try:
    with full_path.open('r') as f:
        reader = csv.reader(f)
        headers = next(reader)
        row1 = next(reader)
        oldest_available_timestamp = datetime.datetime.fromtimestamp(int(row1[0]) // 1000)
except FileNotFoundError:
    print('.csv not found, querying data...')
    candles = get_and_overwrite_data(earliest_required_timestamp, datetime.datetime.now())
    oldest_available_timestamp = datetime.datetime.fromtimestamp(int(candles[-1][0]) // 1000)

if earliest_required_timestamp < oldest_available_timestamp:
    # We have insufficient data in our csv
    get_and_merge_data(earliest_required_timestamp, oldest_available_timestamp)
    

In [15]:
print(data.datetime.datetime(0))
print(data.datetime.datetime(0))
print(data.datetime.datetime(-1))
print(data.datetime.datetime(-2))


2021-06-11 10:50:00
2021-06-11 10:50:00
2021-06-11 10:45:00
2021-06-11 10:40:00


In [59]:
%reset